In [ ]:
!pip install transformers -q
!pip install diffusers["torch"] transformers -q
!pip install accelerate -q
!pip install git+https://github.com/huggingface/diffusers -q
!pip install -U peft -q
!pip install git+https://github.com/huggingface/peft.git -q
!pip install -U peft transformers -q

%pip install --quiet --upgrade mediapy -q

!pip install gradio -q
!pip install safetensors -q
!pip install omegaconf -q
!pip install torchsde -q
!pip install openai==0.28 -q

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
# optional: remove warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import torch
from torch import autocast
import mediapy as media
import gradio as gr
from diffusers import StableDiffusionPipeline
from diffusers import EulerDiscreteScheduler, DPMSolverSDEScheduler
from diffusers.schedulers.scheduling_dpmsolver_multistep import DPMSolverMultistepScheduler
import safetensors
import openai


from PIL import Image
import numpy as np
import os

In [ ]:

openai.api_key = "" # enter your openai api key to start
# def openai_chat(prompt):
#     completions = openai.Completion.create(
#         engine="gpt-3.5-turbo-16k",
#         prompt=prompt,
#         max_tokens=1024,
#         n=1,
#         temperature=0.5,
#     )

#     message = completions.choices[0].text
#     return message.strip()

def openai_chat(chat_prompt):
    completions = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": chat_prompt}
        ]
    )
    # Sonucu strip() ile temizleyin
    return completions.choices[0].message['content'].strip()

In [ ]:
def promptDesign(prompt):

  famous_artists = "Vincent van Gogh, Pablo Picasso, Leonardo da Vinci"
  # famous_artists = "Eric Carle, Dr. Seuss, Beatrix Potter"

  chat_prompt = f"""
  I will give you the 'children's book text' and a formula for the image prompt. Create a formula for the children's book
  text to determine key points of text.
  Here’s a formula for a Stable Diffusion image prompt: An image of [adjective] [subject] [doing action],
  [creative lighting style], detailed, realistic,
  trending on artstation, in the style of {famous_artists}.
  Here's a children's book text to create prompt:
  {prompt}

  Just write the image prompt in English. Only give me prompt, do not write anything different.
  """
  return openai_chat(chat_prompt)

In [ ]:
def image_generator(enable_chatgpt_template, prompt, x_dimension, y_dimension, seed_enabled, seed, enable_inference_steps, num_inference_steps,
                    guidance=7.5):

    defaultSeed = -1
    inference_steps = 0
    # Check if all dimensions are provided
    if prompt and x_dimension and y_dimension:

        # Convert dimensions to integers
        x_dimension = int(x_dimension)
        y_dimension = int(y_dimension)

        # Create diffusion pipeline, load safetensors model file from project folder
        # pipe = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16)
        pipe = StableDiffusionPipeline.from_pretrained("SG161222/Realistic_Vision_V2.0", torch_dtype=torch.float16)
        pipe = pipe.to("cuda" if torch.cuda.is_available() else "cpu")
        pipe.safety_checker = None

        if enable_chatgpt_template:
          prompt = promptDesign(prompt)

        # Apply inference steps if enabled
        if enable_inference_steps:
            inference_steps = num_inference_steps

        if seed_enabled:
          defaultSeed = seed

        pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
        # pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config, use_karras_sigmas=True)
        # pipe.scheduler = DPMSolverSDEScheduler.from_config(pipe.scheduler.config, use_karras_sigmas=True)
        # lora_path = "mehmetalpy/Children_stories"
        # lora_path = "artificialguybr/StoryBookRedmond-V2"
        lora_path = "mehmetalpy/the-kidbook"
        pipe.load_lora_weights(lora_path, weight_name="COOLKIDS_MERGE_V2.5.safetensors")

        generator = torch.Generator("cuda").manual_seed(defaultSeed)
        # prompt = "closeup, a family portrait, momn, dad, kitchen in the background"
        # h=512
        # w=512
        # steps=40
        # guidance=7.5
        lora_weight=1
        # cfg_scale=5
        # num_images=1
        negativePrompt = "ngtvH, ngtv, ngtvB"

        with autocast("cuda"):
          image = pipe(prompt, generator=generator, cross_attention_kwargs = {"scale": lora_weight}, height=y_dimension,
                       width=x_dimension, num_inference_steps=inference_steps, guidance_scale=guidance,
                       negative_prompt=negativePrompt).images # num_images_per_prompt=num_images


        # Return generated and resized image
        return prompt, image[0]

In [ ]:
# prompt ="closeup, a family portrait, momn, dad, kitchen in the background"
# media.show_images([image_generator(prompt, x_dimension=1024, y_dimension=1024, seed_enabled=True, seed=-1, enable_inference_steps=True, num_inference_steps=40, guidance=7.5)])

In [ ]:
# Options for the radio buttons
dimensions = ["512", "640", "768", "1024"]

# Build the Gradio interface
Interface_ChildBook = gr.Interface(
    fn=image_generator,
    inputs=[
        gr.Checkbox(label="Enable ChatGPT Template"),
        gr.Textbox(label="Text Prompt", placeholder="Type your prompt here..."),
        gr.Radio(label="X Dimension", choices=dimensions),
        gr.Radio(label="Y Dimension", choices=dimensions),
        gr.Checkbox(label="Enable Seed"),
        gr.Number(label="Set Seed Number"),
        gr.Checkbox(label="Enable Inference Steps"),
        gr.Slider(label="Set Inference Steps (Optimal: 25-60)"),
        gr.Number(label="Guidance (Optimal: 3.5-9)", value=7.5),
    ],
    outputs=[
        gr.Textbox(label="Prompt"),
        gr.Image(label="Generated Image"),
    ],
    title="Text-to-Image Generator for Children Books",
    theme=gr.themes.Soft(),
    description="Book Images Generator with Stable Diffusion and LoRA Models. If you do not know How to Write LoRA prompt then click Enable ChatGPT Template",
    examples=[
        [False, "closeup, a family portrait, momn, dad, kitchen in the background", "1024", "1024", True, 4128965862, True, 40, 7.5],
        [False, "a Wolf is camping, kid, Desert Scrub", "1024", "1024", True, 3490148812, True, 40, 7.5],
        [False, "a Porcupine is camping, kid, Intertidal Zone", "1024", "1024", True, 3435676572, True, 40, 7.5],
        [False, "a Meerkat is camping, kid, Savannah", "1024", "1024", True, 313220802, True, 40, 7.5],
        [False, "a Chilean girl in a wasteland, explorer suit, alien planet, space, starfield, kid, Rocky Shore", "1024", "1024", True, 3299919458, True, 40, 7.5],
    ],
)

# Start interface
Interface_ChildBook.launch(debug=True)

In [ ]:
gr.close_all()

In [ ]:
# # unload GPU
# with torch.no_grad():
#     torch.cuda.empty_cache()